In [1]:
# This notebook imports legacy publicwhip policy information into yaml files that power new database

from pathlib import Path
import os
import re
import pandas as pd
from datetime import date

while not (Path.cwd() / "pyproject.toml").exists():
    os.chdir("..")

from twfy_votes.internal.db import duck_core
from twfy_votes.apps.legacy.data_sources import duck as tables_duck
from twfy_votes.apps.policies.models import (
    PolicyDirection,
    PartialPolicy,
    PolicyStrength,
    PartialDivision,
    PartialPolicyDecisionLink,
    PolicyStatus,
    StrengthMeaning,
)

duck = await duck_core.get_core()

await duck.compile(tables_duck).run()

In [2]:
query = """
select dream_id as id, description as policy_description from pw_dyn_dreammp
"""

full_desc_df = await duck.compile(query).df()

In [3]:
query = """
select * from pw_dyn_dreamvote order by dream_id, house, division_date, division_number
"""


def get_direction(s: str) -> PolicyDirection:
    match s:
        case "aye" | "aye3":
            return PolicyDirection.AGREE
        case "no" | "no3":
            return PolicyDirection.AGAINST
        case "both":
            return PolicyDirection.NEUTRAL
        case _:
            raise ValueError(f"Unknown vote direction {s}")


def get_strength(s: str) -> PolicyStrength:
    match s:
        case "aye" | "no" | "both":
            return PolicyStrength.WEAK
        case "aye3" | "no3":
            return PolicyStrength.STRONG
        case _:
            raise ValueError(f"Unknown vote strength {s}")


def get_partial(s: pd.Series) -> PartialDivision:
    return PartialDivision(
        chamber_slug=s["house"],
        date=s["division_date"],
        division_number=int(s["division_number"]),
    )


def get_links(s: pd.Series) -> PartialPolicyDecisionLink:
    return PartialPolicyDecisionLink(
        division=s["partial_division"], alignment=s["direction"], strength=s["strength"]
    )


votes_df = await duck.compile(query).df()
votes_df["direction"] = votes_df["vote"].apply(get_direction)
votes_df["strength"] = votes_df["vote"].apply(get_strength)
votes_df["partial_division"] = votes_df.apply(get_partial, axis=1)
votes_df["links"] = votes_df.apply(get_links, axis=1)

# create a dict vote_lookup that maps from dream_id to a list of PolicyDivisionLink objects

vote_lookup = votes_df.groupby("dream_id").agg({"links": list}).to_dict()["links"]

In [4]:
df = pd.read_csv(Path("data", "raw", "pre_2023_policies.csv"))  # type: ignore


def remove_html_tags(s: str) -> str:
    return re.sub(r"<[^>]*>", "", s)


def slugify(s: str) -> str:
    # if more than 5 words, limit to that
    s = " ".join(s.split(" ")[:5]).strip()
    return re.sub(r"[^a-z0-9]+", "-", s.lower())


# delete the multiple_groups column, and then group the data by policy_id so the group column becomes a list of groups, but we also want to keep the label and summary columns

df = (
    df.drop(columns=["multiple_groups"])
    .groupby("policy_id")
    .agg({"group": list, "label": "first", "summary": "first"})
    .reset_index()
    .rename(
        columns={
            "label": "context_description",
            "summary": "highlightable",
            "policy_id": "id",
        }
    )
)

# bring in the long descriptions from the duck database

df = df.merge(full_desc_df, on="id", how="left")

df["name"] = df["context_description"].apply(remove_html_tags)

df

,id,group,context_description,highlightable,policy_description,name
0,363,[health],introducing <b>foundation hospitals</b>,False,foundation hospitals should be introduced,introducing foundation hospitals
1,810,[misc],greater <b>regulation of gambling</b>,False,gambling should not be made more accessible by...,greater regulation of gambling
2,811,"[social, health]",<b>smoking bans</b>,False,all smoking should be banned,smoking bans
3,826,[social],equal <b>gay rights</b>,False,gays and lesbians should have the same rights ...,equal gay rights
4,837,[reform],a <strong>wholly elected</strong> House of Lords,True,the House of Lords should be wholly elected,a wholly elected House of Lords
...,...,...,...,...,...,...
87,6754,[taxation],reducing <b>capital gains tax</b>,False,Capital gains taxes should be reduced.,reducing capital gains tax
88,6757,[foreignpolicy],"<b>military action against <a href=""https://en...",False,The actions of Daesh / ISIL need to be tackled...,military action against ISIL (Daesh)
89,6758,[home],<b>merging police and fire services</b> under ...,False,Police and Crime Cmmrs should have their role ...,merging police and fire services under Police ...
90,6761,[foreignpolicy],<b>UK membership of the EU</b>,False,The UK should remain a member of the EU,UK membership of the EU


In [5]:
policies: list[PartialPolicy] = []

for _, row in df.iterrows():
    policy = PartialPolicy(
        chamber_id="commons",
        status=PolicyStatus.ACTIVE,
        strength_meaning=StrengthMeaning.CLASSIC,
        id=row["id"],
        name=row["name"],
        context_description=row["context_description"],
        highlightable=row["highlightable"],
        policy_description=row["policy_description"],
        group_ids=row["group"],
        decision_links_refs=vote_lookup[row["id"]],
    )
    policies.append(policy)

In [6]:
data_path = Path("data", "policies")
for x in policies:
    x.to_path(data_path / f"{x.id}.yml", x.model_dump_reduced())